In [ ]:
from scipy.optimize import brute
import numpy as np
from datetime import datetime,timedelta
import pandas as pd

In [ ]:
# Load solver
GlobalOptimization = False
import ray

if GlobalOptimization:
    import LearnerGlobalOpt as Learner  # basinhopping global optimization (several times minimize)
else:
    import Learner #minimize

In [ ]:
import ray

In [ ]:
# Initial parameters
dfparam = pd.read_csv("data/param.csv")
dfparam

In [ ]:
dfSP = pd.read_csv("data/dados_municipios_SP.csv")
dfSP

# lista DRSs
DRS = list(dfSP["DRS"].unique())
DRS.remove("Indefinido")

In [ ]:
#@ray.remote
def opt(districtRegion,e0,a0,r0,d0,date,version,wcases,wrec):
    rranges = [slice(1e5,2e6,1e5),slice(-2,2,4),slice(0,1000,1000)]
    optimal = brute(fobjective,        
        ranges=rranges,
        args=(districtRegion,e0,a0,r0,d0,date,version,wcases,wrec), full_output=True, disp=True, finish=None)
    
    return optimal

In [ ]:
from scipy.integrate import odeint

def fobjective(point,districtRegion,e0,a0,r0,d0,date,version,wcases,wrec):
    results=[]
    s0, deltaDate, i0 = point
    
    startNCases = 20
    ratio = 0.1
    version = 1
    cleanRecovered = False
    
    Date = datetime.strptime(date, "%Y-%m-%d")
    end_date = Date + timedelta(days=+int(deltaDate))
    dateStr=end_date.strftime("%Y-%m-%d")
    
    
    predict_range = 60
    
    
    #objective function Odeint solver
    def lossOdeint(point, data, death, s_0, e_0, a_0, i_0, r_0, d_0, startNCases, ratioRecovered, weigthCases, weigthRecov):
        size = len(data)
        beta, beta2, sigma, sigma2, sigma3, gamma, b, mu = point
        def SEAIRD(y,t):
            S = y[0]
            E = y[1]
            A = y[2]
            I = y[3]
            R = y[4]
            D = y[5]
            p=0.2
            # beta2=beta
            y0=-(beta2*A+beta*I)*S+mu*S #S
            y1=(beta2*A+beta*I)*S-sigma*E-mu*E #E
            y2=sigma*E*(1-p)-gamma*A-mu*A #A
            y3=sigma*E*p-gamma*I-sigma2*I-sigma3*I-mu*I#I
            y4=b*I+gamma*A+sigma2*I-mu*R #R
            y5=(-(y0+y1+y2+y3+y4)) #D
            return [y0,y1,y2,y3,y4,y5]

        y0=[s_0,e_0,a_0,i_0,r_0,d_0]
        tspan=np.arange(0, size, 1)
        res=odeint(SEAIRD,y0,tspan,hmax=0.01)

        l1=0
        l2=0
        l3=0
        tot=0

        for i in range(0,len(data.values)):
            if data.values[i]>startNCases:
                l1 = l1+(res[i,3] - data.values[i])**2
                l2 = l2+(res[i,5] - death.values[i])**2
                newRecovered=min(1e6,data.values[i]*ratioRecovered)
                l3 = l3+(res[i,4] - newRecovered)**2
                tot+=1
        l1=np.sqrt(l1/max(1,tot))
        l2=np.sqrt(l2/max(1,tot))
        l3=np.sqrt(l3/max(1,tot))

        #weight for cases
        u = weigthCases  #Brazil US 0.1
        w = weigthRecov
        #weight for deaths
        v = max(0,1. - u - w)
        
        #print(u*l1 + v*l2 + w*l3)
        return u*l1 + v*l2 + w*l3
    
    learner = Learner.Learner.remote(districtRegion, lossOdeint,dateStr, 
                                     predict_range,s0, e0, a0, i0, r0, d0, 
                                     startNCases, ratio, wcases, wrec,
                                     cleanRecovered, version, savedata=False)
    
    results.append(learner.train.remote())
    results = ray.get(results)
    return results

In [ ]:
allDistricts=False

optimal=[]
if allDistricts:
    for i in range(len(DRS))
    #for districtRegion in DRS:
        query = dfparam.query('DRS == "{}"'.format(DRS[i])).reset_index()
        parameters = np.array(query.iloc[:, 2:])[0]
        startdate,predictionrange,s0,e0,a0,i0,r0,d0,START,ratio,wcases,wrec = parameters

        version = 1
        optimal = []
        optimal.append(opt(districtRegion,e0,a0,r0,d0,startdate,version,wcases,wrec)) 
        
        print("DRS = {}\n".format(DRS[i]))
        print("S0 = {}\n".format(optimal[i][0][0]))
        print("Delta Date Days = {}\n".format(optimal[i][0][1]))   
        print("I0 = {}\n".format(optimal[i][0][2]))   
        print("Function Minimum = {}\n".format(optimal[i][1]))
        
        j = query['index'].values[0]
        dfparam.at[j, "s0"] = optimal[i][0][0]
        dfparam.at[j, "i0"] = optimal[i][0][2]
        
else:
    districtRegion= "DRS 01 - Grande São Paulo" #'DRS 08 - Franca' \
    #'DRS 14 - São João da Boa Vista' #'DRS 04 - Baixada Santista' \
    #'DRS 11 - Presidente Prudente' #'DRS 13 - Ribeirão Preto' \
    #'DRS 05 - Barretos' #'DRS 12 - Registro' #'DRS 15 - São José do Rio Preto' \
    #'DRS 10 - Piracicaba'#'DRS 17 - Taubaté'#'DRS 02 - Araçatuba'# \
    #'DRS 03 - Araraquara' #DRS 07 - Campinas'#'DRS 16 - Sorocaba'#'DRS 06 - Bauru' \
    #'DRS 09 - Marília' #"DRS 01 - Grande São Paulo"

    query = dfparam.query('DRS == "{}"'.format(districtRegion)).reset_index()
    parameters = np.array(query.iloc[:, 2:])[0]
    startdate,predictionrange,s0,e0,a0,i0,r0,d0,START,ratio,wcases,wrec = parameters

    version = 1
    optimal = []
    optimal.append(opt(districtRegion,e0,a0,r0,d0,startdate,version,wcases,wrec))
    
    print("DRS = {}\n".format(DRS[i]))
    print("S0 = {}\n".format(optimal[0][0][0]))
    print("Delta Date Days = {}\n".format(optimal[0][0][1]))   
    print("I0 = {}\n".format(optimal[0][0][2]))   
    print("Function Minimum = {}\n".format(optimal[0][1]))
    
    j = query['index'].values[0]
    dfparam.at[j, "s0"] = optimal[0][0][0]
    dfparam.at[j, "i0"] = optimal[0][0][2]

In [ ]:
ray.shutdown()

In [ ]:
dfparam.to_csv("data/param.csv", sep=",", index=False)